In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load the CSV file
file_path = '/content/dataset.csv'  # Adjust the path as needed
data = pd.read_csv(file_path)

# Drop non-numeric columns and separate features and target variable
# Replace problematic values and convert columns to numeric
def safe_convert_to_numeric(series):
    return pd.to_numeric(series, errors='coerce')  # Coerce invalid strings to NaN

X = data.drop(columns=['HEA combinations', 'Phases', 'Unnamed: 20'], errors='ignore')  # Drop specified columns
X = X.apply(safe_convert_to_numeric)  # Convert all columns to numeric
y = data['Phases']  # Target variable

# Relabel target values to sequential integers
y_mapped, unique_labels = pd.factorize(y)

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_mapped, test_size=0.2, random_state=42)

# Define the XGBoost model with hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],           # Number of trees
    'max_depth': [3, 5, 7],                  # Maximum depth of each tree
    'learning_rate': [0.01, 0.1, 0.2],       # Learning rate
    'subsample': [0.8, 1.0],                 # Fraction of samples used per tree
    'colsample_bytree': [0.8, 1.0]           # Fraction of features used per tree
}

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Perform hyperparameter tuning
grid_search.fit(X_train, y_train)

# Get the best model and parameters
best_xgb_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Make predictions on the test set
y_pred_xgb = best_xgb_model.predict(X_test)

# Calculate the confusion matrix
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_xgb, annot=True, fmt='d', cmap='Blues',
            xticklabels=unique_labels, yticklabels=unique_labels)
plt.title('Confusion Matrix - XGBoost')
plt.xlabel('Predicted Phases')
plt.ylabel('True Phases')
plt.show()

# Print accuracy score
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=unique_labels))
